# Language Classification

We will learn to classify phrases as Spanish or English.

First, lets get all the data.

In [2]:
import requests, re

spanish_novels = ['http://www.gutenberg.org/cache/epub/2000/pg2000.txt']
english_novels = ['https://www.gutenberg.org/files/46787/46787-0.txt']

spanish_sentences = [s for novel in spanish_novels 
                     for s in re.sub(r'\s+', ' ', requests.get(novel).text).split('.')[20:-20]]
english_sentences = [s for novel in english_novels
                     for s in re.sub(r'\s+', ' ', requests.get(novel).text).split('.')[20:-20]]

spanish_sentences[:2], english_sentences[:2]

([' AL DUQUE DE BÉJAR, marqués de Gibraleón, conde de Benalcázar y Bañares, vizconde de La Puebla de Alcocer, señor de las villas de Capilla, Curiel y Burguillos En fe del buen acogimiento y honra que hace Vuestra Excelencia a toda suerte de libros, como príncipe tan inclinado a favorecer las buenas artes, mayormente las que por su nobleza no se abaten al servicio y granjerías del vulgo, he determinado de sacar a luz al Ingenioso hidalgo don Quijote de la Mancha, al abrigo del clarísimo nombre de Vuestra Excelencia, a quien, con el acatamiento que debo a tanta grandeza, suplico le reciba agradablemente en su protección, para que a su sombra, aunque desnudo de aquel precioso ornamento de elegancia y erudición de que suelen andar vestidas las obras que se componen en las casas de los hombres que saben, ose parecer seguramente en el juicio de algunos que, continiéndose en los límites de su ignorancia, suelen condenar con más rigor y menos justicia los trabajos ajenos; que, poniendo los oj

We will define a new network for what we want,

In [7]:
from __future__ import print_function
import mxnet as mx
import numpy as np
from mxnet import nd, autograd, sym
from mxnet import gluon

In [60]:
class LastNet(gluon.Block):
    def __init__(self, recurrent_width, forward_width, **kwargs):
        super(LastNet, self).__init__(**kwargs)
        with self.name_scope():
            self._rnn = gluon.rnn.LSTM(recurrent_width)
            self._nn  = gluon.nn.Dense(forward_width)

    def forward(self, x):
        h = self._rnn(x)
        return self._nn(h[h.shape[0]-1])

Now prep the data and the network,

In [71]:
LIMIT = 100
spanish_sentences, english_sentences = (list(filter(lambda s: len(s) > 0, spanish_sentences)),
                                        list(filter(lambda s: len(s) > 0, english_sentences)))

data = ([(s, 1) for s in spanish_sentences][:LIMIT] + 
        [(s, 0) for s in english_sentences][:LIMIT])


ctx = mx.cpu()

net = LastNet(100, 2)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

net.collect_params().initialize(mx.init.Xavier(), ctx=ctx)
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})

Begin training,

In [ ]:
BATCH_SIZE=20
for epoch in range(10):
    print("Epoch", epoch)
    for batch in np.random.choice(len(data), size=(int(len(data)/BATCH_SIZE), BATCH_SIZE)):
        with autograd.record():
            loss = sum(softmax_cross_entropy(net(nd.array([ord(c) for c in data[i][0]]).reshape(shape=(-1, 1, 1))),
                                             nd.array([data[i][1]]).reshape((-1, 1)))
                       for i in batch)
        loss.backward()
        trainer.step(BATCH_SIZE)
        print("\tLoss:", loss.asscalar())

Epoch 0
	Loss: 16.7151
	Loss: 20.8273
	Loss: 13.6166
	Loss: 14.0178
	Loss: 13.3715
	Loss: 13.7383
	Loss: 13.4944
	Loss: 14.0271
	Loss: 17.2844
	Loss: 17.6789
Epoch 1
	Loss: 18.6013
	Loss: 18.5866
	Loss: 14.864
	Loss: 13.9156
	Loss: 14.0498
	Loss: 14.0009
	Loss: 12.9225
	Loss: 13.3744
	Loss: 14.7069
	Loss: 15.0635
Epoch 2
	Loss: 13.5272
	Loss: 14.2686
	Loss: 14.0251
	Loss: 13.7333
	Loss: 15.5724
	Loss: 16.0863
	Loss: 13.9272
	Loss: 13.6959
	Loss: 13.5329
	Loss: 12.7551
Epoch 3
	Loss: 15.1875
	Loss: 14.854
	Loss: 14.6476
	Loss: 14.1461
	Loss: 19.8791
	Loss: 12.1941
	Loss: 15.3345
	Loss: 13.9373
	Loss: 18.5974
	Loss: 13.9196
Epoch 4
	Loss: 14.0716
	Loss: 13.4959
	Loss: 13.7474
	Loss: 15.7465
	Loss: 12.9978
	Loss: 13.941
	Loss: 14.4755
	Loss: 13.4096
	Loss: 14.3536
	Loss: 13.7129
Epoch 5
	Loss: 13.6495
	Loss: 13.4973
	Loss: 13.348
	Loss: 13.5933
	Loss: 13.9643
	Loss: 12.9618
	Loss: 13.7921
	Loss: 13.6252
	Loss: 13.3976
	Loss: 13.2989
Epoch 6
	Loss: 14.8015
	Loss: 15.3817
	Loss: 13.6578
	Lo

In [67]:
for s in english_sentences[:20]:
    if len(s) > 0:
        x = nd.array([ord(c) for c in s]).reshape(shape=(-1, 1, 1))
        pred = net(x)
        pk = nd.exp(pred)/nd.sum(nd.exp(pred))
        print(s[:30], '...', pk.asnumpy().flatten())

 He looks out after a job, and ... [ 0.66601664  0.33398333]
 He has been an ordinary seama ... [ 0.66173398  0.33826602]
 And strangely enough, though  ... [ 0.65979099  0.34020898]
 Before this present War, he w ... [ 0.65617484  0.34382516]
 At the outbreak of hostilitie ... [ 0.66291416  0.33708584]
 It was his experience as “For ... [ 0.66168475  0.33831525]
” In this dangerous and respon ... [ 0.65999079  0.34000921]
 On one occasion, for instance ... [ 0.66685075  0.33314925]
 After eighteen months of this ... [ 0.65578395  0.34421599]
 Boyd Cable was naturally disg ... [ 0.66238678  0.33761325]
 As may be remembered, the Bri ... [ 0.66744977  0.33255026]
” The following letter has jus ... [ 0.65290487  0.34709516]
 Boyd Cable by the publishers, ... [ 0.66605604  0.33394399]
 They feel that it will give t ... [ 0.66070402  0.33929595]
 “ ... [ 0.60131729  0.39868271]
 Many thanks for all the troub ... [ 0.65624148  0.34375852]
 It certainly is odd that Brit ... [ 0.64949811  0.3

In [68]:
for s in spanish_sentences[:20]:
    if len(s) > 0:
        x = nd.array([ord(c) for c in s]).reshape(shape=(-1, 1, 1))
        pred = net(x)
        pk = nd.exp(pred)/nd.sum(nd.exp(pred))
        print(s[:30], '...', pk.asnumpy().flatten())

 AL DUQUE DE BÉJAR, marqués de ... [ 0.67326111  0.32673883]
 Miguel de Cervantes Saavedra ... [ 0.6429202   0.35707983]
 PRÓLOGO Desocupado lector: si ... [ 0.6637482  0.3362518]
 Pero no he podido yo contrave ... [ 0.65791422  0.34208578]
 Y así, ¿qué podrá engendrar e ... [ 0.67113799  0.32886204]
 Acontece tener un padre un hi ... [ 0.66666156  0.33333847]
 Pero yo, que, aunque parezco  ... [ 0.67033619  0.32966381]
 Todo lo cual te esenta y hace ... [ 0.6655066   0.33449343]
 Sólo quisiera dártela monda y ... [ 0.66323876  0.33676127]
 Porque te sé decir que, aunqu ... [ 0.65945518  0.34054479]
 Muchas veces tomé la pluma pa ... [ 0.67230719  0.32769284]
 -Porque, ¿cómo queréis vos qu ... [ 0.67159802  0.32840198]
 De todo esto ha de carecer mi ... [ 0.66077638  0.33922359]
B ... [ 0.54020214  0.45979792]
C ... [ 0.54010892  0.45989111]
, comenzando en Aristóteles y  ... [ 0.65737915  0.34262085]
 También ha de carecer mi libr ... [ 0.66847938  0.33152056]
 En fin, señor y amigo m